In [323]:
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import time



In [324]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

# # Загрузка и предварительная обработка данных
# path = "DataSets/dataframe_benign_brute_xss_sql.csv"
# df = pd.read_csv(path)
# df.columns = [col.lstrip() for col in df.columns]

# label_encoder = LabelEncoder()
# df['Label'] = label_encoder.fit_transform(df['Label'])

# plt.figure(figsize=(68,68))
# cor = df.corr()

# # absolute value of the correlation
# cor_target = abs(cor["Label"])

# # highly correlated features
# relevant_features = cor_target[cor_target>0.01]

# # getting the names of the features
# names = list(relevant_features.keys())

# # dataframe with only highly correlated features
# df_filtered = df[names]
# print(len(df_filtered.columns))

# X = df_filtered.drop('Label', axis=1)
# y = df_filtered['Label']

# print(y.value_counts())



In [325]:
# plt.figure(figsize=(68,68))
# cor = df.corr()

# # absolute value of the correlation
# cor_target = abs(cor["Label"])

# # highly correlated features
# relevant_features = cor_target[cor_target>0.2]

# # getting the names of the features
# names = list(relevant_features.keys())

# dataframe with only highly correlated features
# df_filtered = df[names]
# print(len(df_filtered.columns))

# umerical_columns = df.columns
# df_numeric = df[umerical_columns]
# n=0
# # Iteruj przez każdą kolumnę i oblicz korelację z kolumną 'Label'
# correlations = {}
# for column in df_numeric.columns:
#     correlation = df[column].corr(df['Label'])
#     correlations[column] = correlation

# # Wyświetl korelacje tylko dla kolumn z wartością bezwzględną większą niż 0.2
# for column, correlation in correlations.items():
#     if abs(correlation) > 0.1:
#         n+=1
#         print(column)
# print(n)


##########################

# plt.figure(figsize=(68,68))
# cor = df.corr()

# # absolute value of the correlation
# cor_target = abs(cor["Label"])

# # highly correlated features
# relevant_features = cor_target[cor_target>0.1]

# # getting the names of the features
# names = list(relevant_features.keys())

# # dataframe with only highly correlated features
# df_filtered = df[names]
# print(len(df_filtered.columns))

In [326]:
# Splitting into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(y_train.value_counts())

Label
0    117646
1      1037
3       465
2        13
Name: count, dtype: int64


In [327]:
# smote_enn = SMOTEENN(random_state=42)
# X_resampled, y_resampled = smote_enn.fit_resample(X_train, y_train)

# print(y_resampled.value_counts())

# Расчет половины размера самого большого класса
half_of_largest_class_size = y_train.value_counts().max() // 2

# Определение стратегии ресэмплинга для андерсэмплинга
undersample_strategy = {label: half_of_largest_class_size for label in y_train.unique()}

# Применение SMOTE для оверсэмплинга
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_smote, y_smote = smote.fit_resample(X_train, y_train)

# Применение RandomUnderSampler для андерсэмплинга
under_sampler = RandomUnderSampler(sampling_strategy=undersample_strategy, random_state=42)
X_resampled, y_resampled = under_sampler.fit_resample(X_smote, y_smote)

print(y_resampled.value_counts())


Label
0    58823
1    58823
2    58823
3    58823
Name: count, dtype: int64


In [328]:
results = []

# Function for training and evaluating models
def train_and_evaluate(model, X_train, y_train, X_test, y_test):

    start_time = time.time() # Start time

    # Train the model
    model.fit(X_train, y_train)

    end_time = time.time()  # Finish time
    training_time = end_time - start_time  # Count training time

    # Predict on test data
    y_pred = model.predict(X_test)

    print(classification_report(y_test, y_pred))

    accuracy = accuracy_score(y_test, y_pred)
    print("Точность:", accuracy)

# Evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)
    conf_matrix = confusion_matrix(y_test, y_pred)

# Save results to the results list
    results.append({
        "Model": model.__class__.__name__,
        "Accuracy": accuracy,
        "Precision": report["weighted avg"]["precision"],
        "Recall": report["weighted avg"]["recall"],
        "F1-Score": report["weighted avg"]["f1-score"],
        "Confusion Matrix": conf_matrix,
        "Training-Time": training_time
    })

# List of models for training
models = [
    GaussianNB(),
    LogisticRegression(max_iter=1000, solver='liblinear'),
    RandomForestClassifier(n_estimators=100, random_state=42)
]

# Apply the function to each model
for model in models:
    train_and_evaluate(model, X_resampled, y_resampled, X_test, y_test)

              precision    recall  f1-score   support

           0       1.00      0.39      0.56     50405
           1       0.02      0.07      0.04       470
           2       0.00      0.75      0.00         8
           3       0.26      0.90      0.40       187

    accuracy                           0.38     51070
   macro avg       0.32      0.53      0.25     51070
weighted avg       0.99      0.38      0.55     51070

Точность: 0.3849422361464656
              precision    recall  f1-score   support

           0       1.00      0.82      0.90     50405
           1       0.39      0.33      0.36       470
           2       0.00      0.62      0.00         8
           3       0.19      0.82      0.30       187

    accuracy                           0.82     51070
   macro avg       0.39      0.65      0.39     51070
weighted avg       0.99      0.82      0.90     51070

Точность: 0.8196984531035834
              precision    recall  f1-score   support

           0     

In [331]:
df_results = pd.DataFrame(results)
print(df_results)
# Extracting model names
# model_names = [model.__class__.__name__ for model in models]
# classification_names = ['BENING', 'Brute Force', 'XSS', 'Sql Injection']


# # Creating a DataFrame for each confusion matrix
# conf_matrix_dfs = [pd.DataFrame(matrix, columns=[f'Predicted {i}' for i in range(matrix.shape[1])],
#                     index=[classification_names[i] for i in range(matrix.shape[0])])
#                    for matrix in df_results['Confusion Matrix']]

# # Output each confusion matrix table
# for i, df in enumerate(conf_matrix_dfs):
#     print(f"Confusion Matrix for Model {model_names[i]}:\n")
#     print(df, "\n")
#     print("-" * 50, "\n")

# print(df_results)
# # Plot the results
# ax = df_results.plot(x='Model', y=['Accuracy', 'Precision', 'Recall', 'F1-Score', 'Training-Time'], kind='bar', figsize=(10,6))
# plt.title('Model Comparison')
# plt.ylabel('Scores')

# # Adding vertical values inside the columns
# for p in ax.patches:
#     ax.annotate(f"{p.get_height():.4f}", (p.get_x() + p.get_width() / 2., p.get_height()/1.33), ha='center', va='bottom', color='white', rotation=90)

# plt.show()


                    Model  Accuracy  Precision    Recall  F1-Score  \
0              GaussianNB  0.384942   0.987740  0.384942  0.551357   
1      LogisticRegression  0.819698   0.990964  0.819698  0.896194   
2  RandomForestClassifier  0.994870   0.995555  0.994870  0.995065   

                                    Confusion Matrix  Training-Time  
0  [[19450, 1388, 29460, 107], [0, 35, 58, 377], ...       0.113273  
1  [[41549, 216, 8228, 412], [1, 154, 52, 263], [...      29.099033  
2  [[50398, 5, 0, 2], [1, 304, 0, 165], [2, 3, 2,...      42.181357  
